In [1]:
import numpy as np
import sqlite3
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
conn = sqlite3.connect("database.sqlite")

In [3]:
c = conn.cursor()

### No of players in the IPL country wise

In [4]:
sql = """
select Country.Country_Name, COUNT(Player.Country_Name) AS Number_of_Players
FROM Country
JOIN Player ON Player.Country_Name=Country.Country_Id
GROUP BY Player.Country_Name
ORDER BY NUmber_of_Players Desc
"""
country = pd.read_sql(sql, conn)
country

,Country_Name,Number_of_Players
0,India,262
1,Australia,72
2,South Africa,39
3,New Zealand,22
4,Sri Lanka,20
5,West Indies,19
6,England,14
7,Pakistan,13
8,Bangladesh,5
9,Zimbabwea,2


### Number of matches played in each stadium
 

In [5]:
sql = """
select Venue.Venue_Name, Count(Match.Venue_id) AS Matches_Played, City.City_Name AS Location
from Venue
JOIN Match ON Venue.Venue_Id = Match.Venue_Id
JOIN City ON City.City_Id = Venue.City_Id
GROUP BY Match.Venue_id
ORDER BY Matches_Played DESC
"""
venue = pd.read_sql(sql, conn)
venue

,Venue_Name,Matches_Played,Location
0,M Chinnaswamy Stadium,58,Bangalore
1,Eden Gardens,54,Kolkata
2,Feroz Shah Kotla,53,Delhi
3,Wankhede Stadium,49,Mumbai
4,"MA Chidambaram Stadium, Chepauk",48,Chennai
5,"Rajiv Gandhi International Stadium, Uppal",41,Hyderabad
6,"Punjab Cricket Association Stadium, Mohali",35,Chandigarh
7,Sawai Mansingh Stadium,33,Jaipur
8,Subrata Roy Sahara Stadium,17,Pune
9,Dr DY Patil Sports Academy,17,Mumbai


### Number of matches played in each city

In [6]:
sql = """
select City.City_Name AS City, Count(Match.Venue_id) AS Matches_Played
from City
JOIN Venue ON City.City_Id = Venue.City_Id
JOIN Match ON Venue.Venue_Id = Match.Venue_Id
GROUP BY City.City_Name
ORDER BY Matches_Played DESC
"""
city = pd.read_sql(sql, conn)
city

,City,Matches_Played
0,Mumbai,77
1,Bangalore,58
2,Kolkata,54
3,Delhi,53
4,Chennai,48
5,Chandigarh,42
6,Hyderabad,41
7,Jaipur,33
8,Pune,25
9,Abu Dhabi,20


### Man of the Series, Purple Cap, Orange Cap winners in every season
 

In [7]:
sql = """
SELECT s.Season_Year, mos.Player_Name Man_of_the_series, oc.Player_Name Orange_cap, pc.Player_Name Purple_Cap
FROM Season s
Join Player mos ON  mos.Player_Id = s.Man_of_the_Series
Join Player oc ON  oc.Player_Id = s.Orange_Cap
Join Player pc ON  pc.Player_Id = s.Purple_Cap
"""
query = pd.read_sql(sql, conn)
query

,Season_Year,Man_of_the_series,Orange_cap,Purple_Cap
0,2008,SR Watson,SE Marsh,Sohail Tanvir
1,2009,AC Gilchrist,ML Hayden,RP Singh
2,2010,SR Tendulkar,SR Tendulkar,PP Ojha
3,2011,CH Gayle,CH Gayle,SL Malinga
4,2012,SP Narine,CH Gayle,M Morkel
5,2013,SR Watson,MEK Hussey,DJ Bravo
6,2014,GJ Maxwell,RV Uthappa,MM Sharma
7,2015,AD Russell,DA Warner,DJ Bravo
8,2016,V Kohli,V Kohli,B Kumar


### Total matches won by each team

In [8]:
sql = """
SELECT  Team.Team_Name as Team_Name,  Count(Match.Match_Winner) AS Number_of_wins
FROM Match
JOIN Team 
ON Team.Team_Id  = Match.Match_Winner
GROUP BY Team_Name
ORDER BY Number_of_wins DESC
"""
matches = pd.read_sql(sql, conn)
matches

,Team_Name,Number_of_wins
0,Mumbai Indians,80
1,Chennai Super Kings,79
2,Royal Challengers Bangalore,70
3,Kolkata Knight Riders,68
4,Rajasthan Royals,63
5,Kings XI Punjab,63
6,Delhi Daredevils,56
7,Sunrisers Hyderabad,34
8,Deccan Chargers,29
9,Pune Warriors,12


### Wicket types

In [9]:
sql = """
select o.Out_Name AS Wicket_type, COUNT(w.Kind_Out) AS Total_dissmissals
from Out_Type o
JOIN Wicket_Taken w
ON o.Out_Id = w.Kind_Out
GROUP BY 1
ORDER BY Total_dissmissals DESC
"""
wickets = pd.read_sql(sql, conn)
wickets

,Wicket_type,Total_dissmissals
0,caught,3954
1,bowled,1251
2,run out,697
3,lbw,399
4,stumped,222
5,caught and bowled,187
6,retired hurt,8
7,hit wicket,8
8,obstructing the field,1


### Top 10 Run scorers


In [10]:
sql = """
select p.player_name, SUM(ba.Runs_Scored) Total_Runs
from batsman_scored ba
JOIN Ball_by_ball bl
ON ba.Match_id = bl.Match_Id
   AND ba.Over_id = bl.Over_id
   AND ba.ball_id = bl.ball_id
   AND ba.Innings_No = bl.Innings_No
JOIN player p
ON p.player_id = bl.striker
Group by bl.Striker
ORDER BY SUM(ba.Runs_Scored) DESC
LIMIT 10
"""
runs_scorers = pd.read_sql(sql, conn)
runs_scorers

,Player_Name,Total_Runs
0,SK Raina,4106
1,V Kohli,4105
2,RG Sharma,3874
3,G Gambhir,3634
4,CH Gayle,3447
5,RV Uthappa,3390
6,DA Warner,3373
7,AB de Villiers,3270
8,MS Dhoni,3270
9,S Dhawan,3082


### Top 10 wicket takers

In [11]:
sql = """
select p.player_name, COUNT(w.Ball_id) Total_Wickets
from Wicket_taken w
JOIN Ball_by_ball bl
ON w.Match_id = bl.Match_Id
   AND w.Over_id = bl.Over_id
   AND w.ball_id = bl.ball_id
   AND w.Innings_No = bl.Innings_No
JOIN player p
ON p.player_id = bl.bowler

Group by bl.bowler
ORDER BY COUNT(w.Ball_id) DESC
LIMIT 10
"""
wicket_takers = pd.read_sql(sql, conn)
wicket_takers

,Player_Name,Total_Wickets
0,SL Malinga,159
1,DJ Bravo,137
2,A Mishra,132
3,Harbhajan Singh,128
4,PP Chawla,127
5,R Vinay Kumar,123
6,A Nehra,111
7,R Ashwin,110
8,Z Khan,107
9,DW Steyn,100


### Players with the most Man of the Match

In [12]:
sql = """
select p.player_name, COUNT(m.Man_of_the_Match) as Total_Awards
from Match m
JOIN Player p
ON p.Player_Id = m.Man_of_the_Match
GROUP BY p.player_name
ORDER BY Total_Awards DESC
LIMIT 10

"""
m_o_m = pd.read_sql(sql, conn)
m_o_m

,Player_Name,Total_Awards
0,CH Gayle,17
1,YK Pathan,16
2,AB de Villiers,15
3,DA Warner,14
4,SK Raina,13
5,RG Sharma,13
6,MS Dhoni,12
7,MEK Hussey,12
8,G Gambhir,12
9,AM Rahane,12


### Total runs scored and total wickets taken in each over

In [13]:
sql = """
SELECT Over_Id AS Over_Number, SUM(Runs_Scored) AS Total_Runs
FROM Batsman_Scored
GROUP BY Over_id
"""
runs = pd.read_sql(sql, conn)
runs

,Over_Number,Total_Runs
0,1,6240
1,2,7407
2,3,8176
3,4,8685
4,5,8758
5,6,8755
6,7,7093
7,8,7705
8,9,7906
9,10,7770


In [14]:
sql = """
SELECT Over_Id AS Over_Number, COUNT(Over_Id) AS Total_wickets
FROM Wicket_Taken
GROUP BY Over_Id
"""
wickets = pd.read_sql(sql, conn)
wickets

,Over_Number,Total_wickets
0,1,236
1,2,269
2,3,296
3,4,282
4,5,295
5,6,290
6,7,241
7,8,243
8,9,278
9,10,263
